Setting up the environment

In [9]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

requirements.txt

In [6]:
%pip install transformers
%pip install langchain
%pip install openai
%pip install google-generativeai
%pip install boto3
%pip install anthropic
%pip install langchain-google-genai
%pip install langchain-huggingface
%pip install langchain-openai
%pip install langchain-anthropic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Imports

In [2]:
from langchain.chat_models import init_chat_model

"Smarter Models"

In [31]:
smarter_models = {
  'gemini-2.0-flash': init_chat_model(model="google_genai:gemini-2.0-flash-thinking-exp-01-21"),
  # 'deepseek': init_chat_model(model="perplexity-ai/r1-1776", model_provider="huggingface"),
  'claude-3.5-haiku': init_chat_model(model="anthropic:claude-3-5-haiku-20241022"),
  # 'o1-mini': init_chat_model(model="openai:o1-mini"),
}

In [33]:
from langchain_core.messages import HumanMessage, SystemMessage

task_prompt = "Write python code to determine if a number is even or odd."
system_prompt_generated_knowledge = "You are a teacher to an AI LLM. For the following task, generate a prompt that contains knowledge needed to solve the problem without giving the answer and start directly with the prompt, Task:\n"

messages = [
    SystemMessage(system_prompt_generated_knowledge),
    HumanMessage(task_prompt),
]



In [30]:
stronger_model_responses = {}
for model in smarter_models:
  stronger_model_responses[model] = smarter_models[model].invoke(messages)

for response in stronger_model_responses:
  print(response+':',stronger_model_responses[response].content)

gemini-2.0-flash: Okay, I'm ready to create a prompt to guide an AI LLM to write Python code for determining even or odd numbers. Here's a prompt that provides the necessary knowledge without giving the direct answer:

**Prompt:**

"To determine if a number is even or odd in Python, you need to understand the concept of divisibility by 2.

Think about the following:

1.  **What defines an even number?**  Consider how even numbers relate to division by 2. Specifically, what is the remainder when an even number is divided by 2?

2.  **What defines an odd number?**  Similarly, consider the remainder when an odd number is divided by 2.

3.  **Python's Remainder Operator:** Python has an operator that calculates the remainder of a division. This is often called the 'modulo' operator.  Research or recall what symbol represents the modulo operator in Python and how it works. For example, if you want to find the remainder when 7 is divided by 2, how would you express that in Python using the m

Benchmark Datasets

Prompt Generation and Storage Loop



(TBD) Prompt Library Database (maybe RAG?)

"Weaker" Models

In [32]:
weaker_models = {
  'gemma-2-2b-it': init_chat_model(model="google_genai:gemma-2-2b-it"),
}

Testing Weaker Model performance on benchmarks without scaffolding prompts

In [ ]:
weaker_model_responses = {}
for model in weaker_models:
  weaker_model_responses[model] = weaker_models[model].invoke(task_prompt)

for response in weaker_model_responses:
  print(response+':',weaker_model_responses[response].content)

Testing Weaker Model performance on benchmarks with scaffolding prompt

In [ ]:
weaker_model_with_scaffolding_responses = {}
for model in weaker_models:
  weaker_model_with_scaffolding_responses[model] = weaker_models[model].invoke(messages)

for response in weaker_model_with_scaffolding_responses:
  print(response+':',weaker_model_with_scaffolding_responses[response].content)

Comparing and plotting change in performance